In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from astropy.table import Table

In [ ]:
input1 = Table({ 
    'ID': [f'CL{i}' for i in range(5)],
    'RA': [0.0, 0.0001, 0.00011, 25, 20],
    'DEC': [0.0, 0.0, 0.0, 0.0, 0.0],
    'Z': [0.2, 0.3, 0.25, 0.4, 0.35],
    'MASS': [10**13.5, 10**13.4, 10**13.3, 10**13.8, 10**14],
    'RADIUS_ARCMIN': [1.0, 1.0, 1.0, 1.0, 1.0],
})
input2 = Table({
    'ID': ['CL0', 'CL1', 'CL2', 'CL3'],
    'RA': [0.0, 0.0001, 0.00011, 25],
    'DEC': [0.0, 0, 0, 0],
    'Z': [0.3, 0.2, 0.25, 0.4],
    'MASS': [10**13.3, 10**13.4, 10**13.5, 10**13.8],
    'RADIUS_ARCMIN': [1.0, 1.0, 1.0, 1.0],
})

In [ ]:
display(input1)
display(input2)

In [ ]:
from clevar.catalog import Catalog

In [ ]:
c1 = Catalog('Cat1', id=input1['ID'], ra=input1['RA'], dec=input1['DEC'], z=input1['Z'], mass=input1['MASS'])
c2 = Catalog('Cat2', id=input2['ID'], ra=input2['RA'], dec=input2['DEC'], z=input2['Z'], mass=input2['MASS'])
display(c1.data)
display(c2.data)

In [ ]:
display(c1)
display(c2)

In [ ]:
from clevar.match.proximity import ProximityMatch
from clevar.cosmology import AstroPyCosmology

In [ ]:
mt = ProximityMatch()

In [ ]:
mt_config1 = {'delta_z':.2,
            'match_radius': '1 mpc',
            'cosmo':AstroPyCosmology()}
mt_config2 = {'delta_z':.2,
            'match_radius': '1 arcsec'}
mt.prep_cat_for_match(c1, **mt_config1)
mt.prep_cat_for_match(c2, **mt_config2)

In [ ]:
display(c1.mt_input)
display(c2.mt_input)

In [ ]:
mt.multiple(c1, c2)

In [ ]:
mt.multiple(c2, c1)

In [ ]:
mt.unique(c1, c2, 'angular_proximity')
mt.unique(c2, c1, 'angular_proximity')

In [ ]:
display(c1)
display(c2)

In [ ]:
c1.cross_match()
c2.cross_match()

In [ ]:
display(c1)
display(c2)

In [ ]:
mt.save_matches(c1, c2, out_dir='temp', overwrite=True)

In [ ]:
mt.load_matches(c1, c2, out_dir='temp')
display(c1)
display(c2)